In [ ]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.4 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=c3a0a5813d5f25229491abd867494657b87c098e696ca7367a9d931091e4f9a0
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [ ]:
import py_vncorenlp
import os

if not os.path.exists('/content/vncorenlp'):
    os.mkdir('/content/vncorenlp')



py_vncorenlp.download_model('/content/vncorenlp')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_articles_from_page(url,category):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    for item in soup.select('.title-news a'):
        title = item.get_text(strip=True)
        if title != '':
            articles.append({
                'title': title,
                'category': category
            })
    return articles

def get_articles_from_vnexpress(start_page, end_page):
    all_articles = []
    categorys = ['thoi-su', 'the-gioi', 'kinh-doanh', 'giai-tri', 'the-thao', 'phap-luat', 'giao-duc', 'suc-khoe', 'doi-song', 'du-lich']
    for category in categorys:
        for page in range(start_page, end_page + 1):
            url = ""
            if page == 1:
                url = f'https://vnexpress.net/{category}'
            else:
                url = f'https://vnexpress.net/{category}-p{page}'
            articles = get_articles_from_page(url,category)
            all_articles.extend(articles)

    return all_articles

# Ví dụ thu thập từ trang 1 đến trang 5
articles = get_articles_from_vnexpress(1, 20)

# Lưu dữ liệu vào file CSV
df = pd.DataFrame(articles)
df.to_csv('vnexpress_articles.csv', index=False)


In [ ]:
import py_vncorenlp
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
import random

data = pd.read_csv('vnexpress_articles.csv')

rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir="/content/vncorenlp")


data['title'] = data['title'].apply(lambda x: rdrsegmenter.word_segment(x)[0])

data.to_csv('/content/vnexpress_articles_segmented.csv', index=False)

#data = pd.read_csv('vnexpress_articles_segmented.csv')

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
model = AutoModel.from_pretrained("vinai/phobert-base-v2")

collections = []

category_to_index = {'thoi-su': 0, 'the-gioi': 1, 'kinh-doanh': 2, 'giai-tri': 3, 'the-thao': 4, 'phap-luat': 5, 'giao-duc': 6, 'suc-khoe': 7, 'doi-song': 8, 'du-lich': 9}
index_to_category = {v: k for k, v in category_to_index.items()}

for i in range(len(data)):
    row = data.iloc[i]
    title = row['title']
    category = row['category']
    category_index = torch.tensor([category_to_index[category]])
    encoded_title = torch.tensor([tokenizer.encode(title)])
    with torch.no_grad():
        output = model(encoded_title).last_hidden_state
    collections.append((category_index, output[0,1:-1].unsqueeze(0)))

random.shuffle(collections)

with open('/content/data.pkl', 'wb') as f:
    pickle.dump(collections, f)

with open('/content/category_to_index.pkl', 'wb') as f:
    pickle.dump(category_to_index, f)

with open('/content/index_to_category.pkl', 'wb') as f:
    pickle.dump(index_to_category, f)










/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
import torch.nn as nn
import pickle

class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
import pickle


input_size = 768
hidden_size = 128
num_classes = 10
model = Model(input_size, hidden_size, num_classes)

with open('/content/model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from sklearn.model_selection import train_test_split
from itertools import product

if __name__ == '__main__':
    with open('/content/data.pkl', 'rb') as f:
        data = pickle.load(f)
    train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
    input_size = 768
    hidden_size = 128
    num_classes = 10
    criterion = nn.CrossEntropyLoss()

    optimizers = [optim.SGD, optim.Adam, optim.AdamW]
    learning_rates = [0.001, 0.01]
    num_epochs_list = [7,9,15]

    best_accuracy = 0
    best_params = None

    best_precision = {}



    for opt, lr, num_epochs in product(optimizers, learning_rates, num_epochs_list):
      with open('/content/model.pkl', 'rb') as f:
        model = pickle.load(f)
      if (opt == optim.SGD):
        optimizer = opt(model.parameters(), lr=lr, momentum=0.9)
      else:
        optimizer = opt(model.parameters(), lr=lr)

      for epoch in range(num_epochs):
          for target, input in train_data:
              optimizer.zero_grad()
              output = model(input)
              loss = criterion(output, target)
              loss.backward()
              optimizer.step()

          print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

      with open('model.pkl', 'wb') as f:
          pickle.dump(model, f)

      model.eval()
      correct = 0
      total = len(test_data)
      precision = {0: [0,0], 1: [0,0], 2: [0,0], 3: [0,0], 4: [0,0], 5: [0,0], 6: [0,0], 7: [0,0], 8: [0,0], 9: [0,0]}
      with torch.no_grad():
          for target, input in test_data:
              output = model(input)
              _, predicted = torch.max(output, 1)
              correct += (k := (predicted == target).sum().item())
              precision[target.item()][1] += 1
              precision[predicted.item()][0] += k

      accuracy = 100 * correct / total
      print(f'Accuracy: {accuracy:.2f}%')
      if accuracy > best_accuracy:
          best_accuracy = accuracy
          best_params = (opt, lr, num_epochs)
          best_precision = precision


    print(f'Best Accuracy: {best_accuracy:.2f}%')
    print(f'Best Parameters: {best_params}')
    print(f'Best Precision: {best_precision}')
    print([round(value[0] * 100/ value[1]) for value in best_precision.values()])



Epoch 1/7, Loss: 1.0735
Epoch 2/7, Loss: 0.5593
Epoch 3/7, Loss: 0.2108
Epoch 4/7, Loss: 0.4330
Epoch 5/7, Loss: 0.1361
Epoch 6/7, Loss: 0.2784
Epoch 7/7, Loss: 0.0038
Accuracy: 83.10%
Epoch 1/9, Loss: 1.0735
Epoch 2/9, Loss: 0.5593
Epoch 3/9, Loss: 0.2108
Epoch 4/9, Loss: 0.4330
Epoch 5/9, Loss: 0.1361
Epoch 6/9, Loss: 0.2784
Epoch 7/9, Loss: 0.0038
Epoch 8/9, Loss: 0.1782
Epoch 9/9, Loss: 0.0013
Accuracy: 85.29%
Epoch 1/15, Loss: 1.0735
Epoch 2/15, Loss: 0.5593
Epoch 3/15, Loss: 0.2108
Epoch 4/15, Loss: 0.4330
Epoch 5/15, Loss: 0.1361
Epoch 6/15, Loss: 0.2784
Epoch 7/15, Loss: 0.0038
Epoch 8/15, Loss: 0.1782
Epoch 9/15, Loss: 0.0013
Epoch 10/15, Loss: 0.0027
Epoch 11/15, Loss: 0.0003
Epoch 12/15, Loss: 0.0005
Epoch 13/15, Loss: 0.0001
Epoch 14/15, Loss: 0.0001
Epoch 15/15, Loss: 0.0000
Accuracy: 84.80%
Epoch 1/7, Loss: 1.0197
Epoch 2/7, Loss: 2.9933
Epoch 3/7, Loss: 0.0234
Epoch 4/7, Loss: 0.7517
Epoch 5/7, Loss: 0.3441
Epoch 6/7, Loss: 0.0024
Epoch 7/7, Loss: 0.0018
Accuracy: 83.99%

In [ ]:
from google.colab import files
files.download('/content/data.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>